In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time, os
from datetime import datetime
import random
import pickle
from time import sleep
from collections import defaultdict
import seaborn as sns
import statistics

In [6]:
with open('pickles/cleaned_210115_data_laundry_revised.pickle','rb') as read_file:
    df = pickle.load(read_file)

In [7]:
df.shape

(1209, 22)

---

selenium for walk scores by neighborhood -- didn't end up using since walkscore.com's neighborhood  
designations are different than those of compass.com

---

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

*** thoughts .. maybe find walk score for neighborhood or zip code instead of address

In [4]:
city_list = list(df.city.unique())
city_list[:3]

['Oakland', 'Emeryville', 'San Leandro']

In [5]:
city_tags = [city.replace(' ', '_') for city in city_list]
base_url = 'https://www.walkscore.com/CA/'
urls = [base_url + city_tag for city_tag in city_tags]
city_dict = dict(zip(urls, city_list))
# response = requests.get(url)
# print(response)
# search_page = response.text
# search_page_soup = BeautifulSoup(search_page, 'lxml')

In [62]:
def assemble_score_dict(neighborhood_val, score_val):
    scores['city'].append(city_dict[url])
    scores['neighborhood'].append(neighborhood_val)
    scores['walk'].append(score_val)

In [70]:
scores = defaultdict(list)
for url in urls[2:5]:    
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)

    time.sleep(4)
    try:
        button = driver.find_element_by_id('read-more-link')
        button.click()
    except:
        print('button not found')

    time.sleep(4)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    driver.quit()

    all_text = soup.text.lower()
    
    try:
        for i in soup.find('tbody'):
            neighborhood_name = i.find('a')
            if neighborhood_name != -1:
                assemble_score_dict(neighborhood_name.text, neighborhood_name.findNext().text)
    except TypeError:
        try:
            ws_idx = all_text.find('has an average walk score of ') + len('has an average walk score of ')
            avg_score = all_text[ws_idx: ws_idx+2]
            assemble_score_dict('N/A', avg_score)

        except:
            assemble_score_dict('N/A', 'N/A')

button not found


In [67]:
    scores = defaultdict(list)

    all_text = soup.text.lower()
    try:
        for i in soup.find('tbody'):
            neighborhood_name = i.find('a')
            if neighborhood_name != -1:
                assemble_score_dict(neighborhood_name.text, neighborhood_name.findNext().text)
    except TypeError:
        try:
            ws_idx = all_text.find('has an average walk score of ') + len('has an average walk score of ')
            avg_score = all_text[ws_idx: ws_idx+2]
            assemble_score_dict('N/A', avg_score)

        except:
            assemble_score_dict('N/A', 'N/A')

walk_data = pd.DataFrame.from_dict(scores)

---

**walk score by individual address**

---

In [14]:
street_type = {'road':'rd', 'real':'real', 'boulevard': 'blvd', 'court': 'ct', 'avenue': 'ave', 'lane': 'ln', 'street': 'st', 'drive': 'dr', 
               'way': 'way', 'parkway':'pkwy', 'common':'common', 'parks': 'parks', 'terrace': 'terrace', 'st.': 'st',
              'place': 'pl', 'circle': 'cir', 'loop':'loop', 'estates': 'estates', 'plaza': 'plaza', 'chavez': 'chavez',
              'broadway':'broadway', 'park':'parks', 'east': 'ave-e', 'alley': 'alley', 'shipley':'shipley', 'mar':'mar',
              'prenda': 'prenda'}
addresses = [address.lower().split(' ') for address in list(df.address_1)]
try:    
    for address in addresses:
        address[-1] = street_type[address[-1]]
except KeyError:
    pass
url_pt1 = 'https://www.walkscore.com/score/'
urls_pt2 = [('-').join(address) for address in addresses]
urls_pt3 = ['-'+city.lower().replace(' ', '-')+'-ca-'+zip_code for city, zip_code in zip(list(df.city), list(df.zip))]
urls = [url_pt1 + urls_pt2[i] + urls_pt3[i] for i in range(0, len(urls_pt2))]

In [15]:
address_soups = []
for url in urls:
    response = requests.get(url)
    print(response, end = ' ')
    address_page = response.text
    address_page_soup = BeautifulSoup(address_page, 'lxml')
    address_soups.append(address_page_soup)
len(address_soups)

<Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200

1209

In [17]:
urls[-1]

'https://www.walkscore.com/score/140-west-main-avenue-morgan-hill-ca-95037'

In [8]:
score_text = str(address_soups[550].find('div', class_='block-header-badge score-info-link'))

In [23]:
walk_scores = []

for soup in address_soups:  
    score_text = str(soup.find(
        'div', class_='block-header-badge score-info-link'))

    score_idx = score_text.find('score/')+len('score/')
    score_str = score_text[score_idx:score_idx+3].replace('.','').replace('p','')
    try:
        score_int = int(score_str)
    except ValueError:
        #integer flag for score not found
        score_int = 1000
    walk_scores.append(score_int)
    
median_walk_score = int(statistics.median([i for i in walk_scores if i < 101]))
walk_score_imp = [i if i < 101 else median_walk_score for i in walk_scores]

In [24]:
pd.Series(walk_score_imp).unique()

array([  3,  87,  92,  55,   6,  70,  89,  97,  69, 100,  65,  86,  99,
        88,  74,  96,  66,  54,  94,  27,  91,  77,  57,  33,  35,  47,
        78,  28,  61,  84,  81,  50,  82,  19,  30,  37,  16,  62,  39,
         9,  93,  67,  24,  22,  41,   1,  98,  14,  48,  52,  83,  75,
        58,  12,  21,  73,  56,  72,  44,  63,  13,   8,  95,  76,  90,
        85,  64,  68,  45,  80,  79,  25,  29,   2,   5,   7,  46,  60,
        71,   4,  40,  38,  32,  59,  15,  20,  17,  53,  10,  51,   0,
        36,  49,  42,  34,  31,  43,  26,  18,  23])

In [26]:
df['walk_score'] = walk_score_imp
df.head(2)

city    zip neighborhood           address_1 address_2   price  beds  \
0  Oakland  94618    Parkwoods  240 Caldecott Lane  Unit 314  2500.0   1.0   
1   Albany  94706     Hillside  625 Jackson Street    Unit 2  2850.0   2.0   

   baths  half_baths  sq_footage  ... year_built          county  \
0    1.0         0.0         773  ...     1996.0  Alameda County   
1    2.0         0.0        1112  ...     1972.0  Alameda County   

    laundry_type  laundry_snippet    parking_spots  \
0  laundry_avail    edlaundryappl          parking   
1  laundry_avail    onlaundryinbu  parking_unknown   

                            parking_snippet has_address_2 baths_incl_half  \
0  raditionalparking typedesignated on site             1             1.0   
1  esignated parking space in the carport.              1             2.0   

  clean_mls_types  walk_score  
0       apartment           3  
1           other          87  

[2 rows x 23 columns]

In [27]:
with open('pickles/cleaned_210115_data_laundry_revised_add_ws.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)